# Creating epochs and generating evoked responses (ERP/ERF)

In [ ]:
import pathlib
import matplotlib

import mne
import os
import mne_bids

matplotlib.use('Qt5Agg')

In [ ]:
# pre-process and filter raw
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject='01',
                              session='01',
                              task='audiovisual',
                              run='01',
                              datatype='meg',
                              root=bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=0.1, h_freq=40)
events, event_id = mne.events_from_annotations(raw)
raw.plot()

In [ ]:
event_id

## Epoch the data

To create epochs we need to specifiy the beginning and end of the epoch as tmin and tmax (for each epoch). This is always relative to an event onset. Should we also apply some form of baseline correction? we can add baseline **(None, 0)**. Here **None** means from the very beginning of the epoch and **0** means until the event onset.  

In [ ]:
tmin = -0.3
tmax = 0.5
baseline = (None, 0) 

epochs = mne.Epochs(raw,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    baseline=baseline,
                    preload=True)
epochs

In [ ]:
epochs.plot()

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Create epochs starting 250 ms before the stimulus onset and ending 800 ms after stimulus onset, and apply baseline correctin with a baseline period ranging from -200 to 0 ms.</li>
    </ul>
</div>

In [ ]:
tmin = -0.25
tmax = 0.8
baseline = (-0.20, 0) 

epochs2 = mne.Epochs(raw,
                    events=events,
                    event_id=event_id,
                    tmin=tmin,
                    tmax=tmax,
                    baseline=baseline,
                    preload=True)
epochs2

In [ ]:
epochs2.plot()

## Selecting epochs based on experimental conditions

In [ ]:
epochs['Auditory/Right']

### And if we want to plot only a subset of epochs or epochs based on specific codnitions:

In [ ]:
epochs['Auditory/Right'].plot()

In [ ]:
epochs['Auditory'].plot()

In [ ]:
epochs['Left']

In [ ]:
epochs['Visual'].plot_image()

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Extract all epochs with a "Right" condition.</li>
    </ul>
</div>

In [ ]:
epochs['Auditory/Right']

In [ ]:
epochs['Visual/Right']

In [ ]:
epochs['Auditory/Right'].plot_image(picks= 'eeg')

In [ ]:
# another way to visualise the above plot:
(epochs['Auditory/Right']
 .copy()
 .pick_types(meg=False, eeg=True)
 .plot_image())

In [ ]:
epochs['Visual/Right'].plot_image(picks= 'eeg')

## Saving epochs

In [ ]:
epochs.save(pathlib.Path('out_data') / 'epochs_epo.fif', 
            overwrite=True)

## Creating evoked data

In [ ]:
evoked_auditory = epochs['Auditory'].average()
evoked_visual = epochs['Visual'].average()

In [ ]:
evoked_auditory.plot(spatial_colors=True)

In [ ]:
evoked_auditory.plot_topomap(ch_type='mag')

### We can also specify which time points we want to visualise:

In [ ]:
evoked_auditory.plot_topomap(ch_type='mag', times=[0, 0.050, 0.100, 0.150, 0.200])

In [ ]:
evoked_auditory.plot_joint(picks='mag')

In [ ]:
mne.viz.plot_compare_evokeds([evoked_auditory, evoked_visual], picks='mag')

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Plot a GFP comparison for the "Visual/Left" and "Visual/Right" conditions of the EEG data.</li>
    </ul>
</div>

In [ ]:
evo_visual_left = epochs['Visual/Left'].average()
evo_visual_right = epochs['Visual/Right'].average()

In [ ]:
mne.viz.plot_compare_evokeds([evo_visual_left, evo_visual_right], picks='eeg')

## Saving evoked data

In [ ]:
mne.write_evokeds(fname=pathlib.Path('out_data') / 'evokeds_ave.fif',
                  evoked=[evoked_auditory, evoked_visual])

## Reading evoked data

In [ ]:
evokeds = mne.read_evokeds(fname=pathlib.Path('out_data') / 'evokeds_ave.fif')
evokeds

In [ ]:
evokeds[0]

In [ ]:
evoked = mne.read_evokeds(fname=pathlib.Path('out_data') / 'evokeds_ave.fif',
                          condition='0.50 * Visual/Left + 0.50 * Visual/Right')
evoked